# 연습 4 - 양자 화학. 전자 구조 계산

## 배경

양자 컴퓨팅은 고성능 컴퓨팅에 혁명을 일으킬 수 있는 가능성을 지니고 있으며, 고전적 컴퓨팅의 현재 기능을 넘어 다양한 분야의 혁신을 끌어낼 수 있는 엄청난 잠재력을 보유하고 있습니다. 화학은 양자 컴퓨팅의 가장 유망한 응용 분야 중 하나입니다. 특히, 화학 반응을 더 잘 이해하고 새로운 의약품 및 재료를 설계하기 위해 분자의 전자 구조를 시뮬레이션하는 것은 멀지 않은 미래의 양자 컴퓨터에 대한 흥미로운 응용 분야 중 하나입니다.

VQE(Variational Quantum Eigensolver)는 단기적(near-term) 양자 컴퓨터를 사용하는 양자 화학의 주요 핵심 알고리즘입니다. 이 것은 분자 시스템의 바닥 상태 에너지(최저 에너지)를 찾는 데 사용되는 하이브리드 양자-고전 알고리즘입니다. 이 알고리즘은 Alberto Peruzzo와 동료들[1]에 의해 2014년에 처음 발표되었으며 2017년 IBM Quantum에서 작은 분자들의 바닥 상태 에너지를 시뮬레이션하는 데에 사용되었습니다[2]. 바닥 상태 에너지를 얻으면 화학 반응을 이해하고 연구할 수 있으며 다양한 분자 특성에 대한 추가적인 통찰력을 얻을 수 있습니다.  예를 들어 원자핵에 가해지는 정확한 힘은 화학 시스템에서 시간이 지남에 따라 일어나는 것을 탐구하기 위해 분자 역학 시뮬레이션을 실행하는 것에 사용할 수 있습니다.[3]

VQE 알고리즘은 ansatz라고 불리는 매개 변수화된 양자 회로와, 가장 낮은 에너지를 얻을 때까지 고전적인 최적화기로 반복적으로 업데이트되는 조정 가능한 매개 변수 집합으로 구성됩니다. 고전적 알고리즘에 비해 VQE의 장점은 양자 처리 장치가 고전적 컴퓨터보다 문제의 파동 함수를 더 효율적으로 표현하고 저장할 수 있다는 점에서 기인합니다.

qEOM(quantum Equation Of Motion, 양자 운동 방정식) 알고리즘은 분자의 들뜸 에너지(바닥 상태와 들뜬 상태 에너지의 차이, 전자 전이 에너지라고도 함)를 찾기 위한 VQE의 확장된 형태입니다. [4]

이 연습에서는 VQE 및 qEOM을 사용하여 물 분자를 시뮬레이션하여 Qiskit Nature에서 사용할 수 있는 다양한 도구로 바닥 및 들뜬 상태 에너지를 계산합니다.

### 참고문헌

1. Peruzzo, Alberto, et al. "A variational eigenvalue solver on a photonic quantum processor." Nature communications 5.1 (2014): 1-7.
2. Kandala, Abhinav, et al. "Hardware-efficient variational quantum eigensolver for small molecules and quantum magnets." Nature 549.7671 (2017): 242-246.
3. Sokolov, Igor O., et al. "Microcanonical and finite-temperature ab initio molecular dynamics simulations on quantum computers." Physical Review Research 3.1 (2021): 013125.
4. Ollitrault, Pauline J., et al. "Quantum equation of motion for computing molecular excitation energies on a noisy quantum processor." Physical Review Research 2.4 (2020): 043140.

추가로,  VQE에 대한 리뷰는: Cerezo, M., et al. "Variational quantum algorithms." Nat Rev Phys 3, 625–644 (2021).

## 소개

잡음이 많은 양자 컴퓨터에서 VQE 알고리즘을 실행할 때 가장 중요한 작업 중 하나는 양자 리소스(큐비트 수, 양자 게이트 및 측정값)를 최소화하여 계산 비용을 줄이는 것입니다. 이는 결과적으로 계산 오류를 최소화합니다. 이 연습의 목표는 바닥 상태를 정확하게 표현할 수 있는 가장 작은 최적의 양자 회로를 구성하는 것입니다.

<div class="alert alert-block alert-success">
<b>목표</b>
    
먼저 Qiskit Nature를 사용하여 전자 구조 문제를 정의하는 방법을 배웁니다. 그런 다음 물 분자의 바닥 상태를 근사화하기 위한 가장 효율적인 Ansatz를 찾을 것입니다. 마지막으로 바닥 상태 결과를 사용하여 들뜸 상태들을 계산해 봅니다.
    
<b>계획</b>
    
**파트 1: 물 분자를 위한 VQE:** VQE에 익숙해지고 statevector 시뮬레이션을 실행하여 최상의 조합을 선택합니다.

**파트 2: 물 분자에 대한 qEOM:** 바닥 상태 계산을 확장하여 이 물 분자의 전자 전이 에너지를 찾습니다. 양자 알고리즘 결과를 고전적으로 얻은 결과와 비교하는 방법을 배웁니다.
</div>

<div class="alert alert-block alert-danger">
이어지는 내용은 VQE 시뮬레이션의 배경 이론에 대한 소개입니다. 도전 과제를 수행하기 위해 전체를 이해할 필요는 없습니다. 겁먹지 말아요!
</div>


## 이론

### 양자 및 고전

다음은 VQE를 사용한 분자 시뮬레이션이 양자 컴퓨터에서 수행되는 방식을 나타내는 일반적인 작업 흐름도입니다.

<img src="resources/workflow.png" width="600/">

하이브리드 양자-고전 접근 방식의 핵심 아이디어는 **CPU(고전 처리 장치)** 와 **QPU(양자 처리 장치)** 가 가장 잘할 수 있는 작업을 구분하는 것입니다. CPU는 에너지를 계산하기 위해 측정해야 하는 항을 나열하고 회로 매개 변수를 최적화합니다. QPU는 시스템의 양자 상태를 나타내는 양자 회로를 구현하고 에너지를 측정합니다.

예를 들어, **CPU** 는 문제를 다시 표현하고 단순화하는 데 도움이 되는 함수 집합(오비탈이라고도 함)을 찾는 데 사용할 수 있습니다. 실제로 이러한 오비탈은 [Hartree-Fock(HF) 방법](https://en.wikipedia.org/wiki/Hartree%E2%80%93Fock_method#:~:text=In%20computational%20physics%20and%20chemistry,system%20in%20a%20stationary%20state.) 을 통해 찾을 수 있습니다. 이 방법에 대한 추가 정보는 아래에 제공됩니다.

HF 계산 후 해밀토니언은 결정된 오비탈들을 기저(basis)로 표현됩니다. 그런 다음 페르미온에서 큐비트로의 변환을 사용하여 큐비트 해밀토니언에 매핑됩니다(아래 해밀토니언 섹션 참조). 큐비트의 수를 줄이거나 Ansatz 회로를 짧게 만들기 위해 시스템의 속성을 추가로 분석하여 사용할 수 있습니다.

- Z2 대칭 및 2큐비트 축소에 대해서는 [Bravyi *et al* , 2017](https://arxiv.org/abs/1701.08213v1) 을 참조하십시오.
- 얽힘 구축(entanglement forging)에 대해서는 [Eddins *et al.* , 2021](https://arxiv.org/abs/2104.10220v1) .
- 조정가능한 ansatz(Adaptive ansatz)에 대해서는 [Grimsley *et al.* , 2018](https://arxiv.org/abs/1812.11173v2) , [Ratew *et al.* ,2019](https://arxiv.org/abs/1910.09694) , [Tang *et al.* ,2019](https://arxiv.org/abs/1911.10205) . 양자 회로를 단축하는 방법을 찾기 위해 이 논문들의 아이디어를 사용할 수 있습니다.

**QPU** 는 각도 $\vec\theta$로 매개변수화된 양자 회로(아래 Ansatzes 섹션 참조)를 구현하는 데 사용됩니다. 목표는 정확한 바닥 상태 파동함수에 가장 근접한(즉, 에너지를 최소화하는) 이러한 각도 값의 조합을 찾는 것입니다. 그런 다음 QPU에서 준비된 시험 파동 함수에서 서로 다른 연산자의 기대값을 측정할 수 있습니다.

이어지는 내용에서는 VQE 및 qEOM 알고리즘의 각 구성 요소의 수학적 세부 내용을 조금 더 자세히 설명합니다. [VQE에 대한 비디오 에피소드](https://www.youtube.com/watch?v=Z-A6G0WVI9w) 를 보는 것도 도움이 될 것입니다.

### Hartree-Fock

Hartree-Fock 이론은 시스템의 다전자(many-electron) 파동 함수가 단일 전자 분자 오비탈로 구성된 단일 슬레이터 행렬식(Slater determinant)으로 표현될 수 있다는 가정에 기반합니다.

전자는 구별할 수 없는 페르미온입니다. 이것은 두 전자의 자리바꿈(permutation)이 파동함수의 부호를 변경해야 함을 의미합니다(이것은 파울리 배타 원리라고 하는 정말 중요한 제약 조건이며, 주기율표의 특정 구조의 이유입니다).

전자 파동 함수는 언제나 다음과 같은 전개형으로 쓸수 있습니다.

$$\begin{equation} \psi_{\text{elec}}(\mathbf{r}_1, \mathbf{r}_2, ...,\mathbf{r}_n) = \sum_{m_1, m_2,...,m_N} C_{m_1, m_2, ..., m_N} | \phi_{m_1}(\mathbf{r}_1) \phi_{m_2}(\mathbf{r}_2) ... \phi_{m_n}(\mathbf{r}_n) | \end{equation}$$

여기서 $C_{m_1, m_2, ..., m_N}$는 복소수 계수, $| \phi_{m_1}(\mathbf{r}_1) \phi_{m_2}(\mathbf{r}_2) ... \phi_{m_n}(\mathbf{r}_n) |$는 슬레이터 행렬식이고 $ \phi_{m_i}(\mathbf{r_i})$는 단일전자 함수입니다. 이러한 단일 전자 함수를 <b>분자 오비탈(MO)</b> 라고 합니다. MO는 유명한 <b>Hartree-Fock(HF)</b> 방법으로 계산됩니다. 일반적으로 <b>원자 오비탈(AO)</b> 로 확장됩니다.

$$\begin{equation} \phi(\mathbf{r}, s) = \sum_n D_{mn}\chi_n(\mathbf{r})\otimes s \end{equation}$$

($s$는 스핀 변수 $s \in { \alpha, \beta }$입니다). MO는 반대 스핀(스핀 업($\alpha$) 또는 스핀 다운($\beta$))의 두 가지 전자를 가질 수 있으며 채워지거나(점유) 비어 있을 수 있습니다(가상). 분자 오비탈의 이러한 스핀 성분은 Spin Orbital(SO)이라고도 합니다. Jordan-Wigner 매핑을 사용하여 양자 컴퓨터에서 화학 문제를 매핑할 때 하나의 큐비트는 하나의 스핀 오피탈를 나타냅니다.

Hartree-Fock 방법은 계수 $D_{mn}$를 최적화하여 전자 파동함수를 에너지를 최소화 하는 <b>단일 슬레이터 행렬식</b> ($\psi_{\text{elec}}(\mathbf{r}_1, \mathbf{r}_2 , ...,\mathbf{r}_n) = | \phi_{m_1}(\mathbf{r}_1) \phi_{m_2}(\mathbf{r}_2) ... \phi_{m_n}(\ mathbf{r}_n) |$)으로 표현하는 것입니다. 이것은 종종 전자의 슈뢰딩거 방정식의 해에 대한 더 나은 근사치를 계산하기 위한 보다 정교한 이론적 방법들의 훌륭한 계산 출발점으로 사용되곤 합니다. Hartree-Fock 방법은 에너지를 최소화하고 이 "최상의 단일 행렬식"를 제공하는 MO 세트를 제공합니다.

AO는 원자를 중심으로 하는 기능으로 <b>기저 집합(basis set)</b> 을 구성하는 함수의 관점에서 자체적으로 확장됩니다. 양자 화학에서 사용되는 기저 함수는 원자에 상대적인 전자 좌표에서 다항식 함수 $P_l$를 곱한 원자를 중심으로 하는 가우스 함수의 합으로 간주됩니다.

$$\begin{equation} \chi_n(\mathbf{r}) = \sum_l N_l^n \exp\big( - \alpha_l^n (|\mathbf{r} - \mathbf{R}_I^n|^2)\big) P_l(\mathbf{r} - \mathbf{R}_I^n) \end{equation}$$

$\alpha_l^n$의 선택과 다항식의 차수에 따라 <b>기저 집합</b> 이 결정됩니다(보통 다양한 이름을 사용합니다. 유명한 것은 6-31G, cc-pVQZ, TZ2P 등등이 있습니다).

기저 집합(입자 수와 함께)는 총 AO 수를 결정합니다. 동일한 분자에 대해 다른 기저 세트는 다른 수의 AO로 이어지며 MO 수를 바꾸는 것으로 이어집니다.

사용된 기저 함수의 유형도 정확도에 영향을 미칩니다. 단일 기저 함수가 알 수 없는 함수를 더 잘 재현할 수 있을수록, 주어진 수준의 정확도를 달성하는 데 필요한 기저 함수는 더 적어집니다. 기저가 작을수록 MO의 표현이 좋지 않습니다.

### 분자 해밀토니언

시스템의 해밀토니언은 해당 시스템의 총 에너지에 해당하는 연산자입니다. 분자 해밀토니언은 다음과 같이 정의됩니다:<br>$$ \hat{H} =\sum_{r s} h_{r s} \hat{a}_{r}^{\dagger} \hat{a}_{s} +\frac{1}{2} \sum_{p q r s} g_{p q r s} \hat{a}_{p}^{\dagger} \hat{a}_{q}^{\dagger} \hat{a}_{r} \hat{a}_{s}+E_{N N} $$ where $$ h_{p q}=\int \phi_{p}^{*}(r)\left(-\frac{1}{2} \nabla^{2}-\sum_{I} \frac{Z_{I}}{R_{I}-r}\right) \phi_{q}(r) $$ $$ g_{p q r s}=\int \frac{\phi_{p}^{*}\left(r_{1}\right) \phi_{q}^{*}\left(r_{2}\right) \phi_{r}\left(r_{2}\right) \phi_{s}\left(r_{1}\right)}{\left|r_{1}-r_{2}\right|} $$

$h_{rs}$ 및 $g_{pqrs}$는 1체/2체 적분(HF 계산 결과의 오비탈로 표현된 해밀토니언 항)이고 $E_{NN}$는 핵 반발 에너지입니다. 1체형 적분은 전자의 운동 에너지와 핵과의 상호 작용을 나타냅니다. 2체 적분은 전자-전자 상호작용을 나타냅니다. $\hat{a}_{r}^{\dagger} / \hat{a}_{r}$ 연산자는 스핀 오비탈 $r$에서 전자를 생성/소멸합니다. 이 항들은 양자 컴퓨터에서 기대값을 측정할 수 있도록 파울리 연산자에 매핑해야 합니다. VQE는 전자 에너지를 최소화하므로 총 에너지를 계산하려면 핵 반발 에너지 $E_{NN}$를 계산하여 추가해야 합니다.

### 해밀토니언 매핑

$ h_{rs}$ 및 $g_{pqrs}$ 텐서의 0이 아닌 모든 행렬 요소에 대해 다음과 같은 페르미온에서 큐비트로의 변환을 사용하여 해당하는 Pauli 문자열(Pauli 연산자의 텐서 곱)을 구성할 수 있습니다. 예를 들어, 오비탈 $r = 3$에 대한 Jordan-Wigner 매핑으로 다음 Pauli 문자열을 얻습니다:<br>$$ \hat a_{3}^{\dagger}= \hat \sigma_z \otimes \hat \sigma_z \otimes\left(\frac{ \hat \sigma_x-i \hat \sigma_y}{2}\right) \otimes 1 \otimes \cdots \otimes 1 $$,  여기서 $\hat \sigma_x, \hat \sigma_y, \hat \ sigma_z$는 잘 알려진 Pauli 연산자입니다. $\hat \sigma_z$ 연산자의 텐서 곱은 페르미온의 반-교환(anti-commutation) 관계를 구현하기 위해 사용됩니다. 물 분자의 14개 스핀 오비탈과 약 14개 큐비트 사이의 Jordan-Wigner 매핑은 다음과 같습니다.

<img src="resources/mapping.png" width="600/">

그런 다음, 하나는 단순히 1체/2체 여기를 대체합니다(예: $\hat{a}_{r}^{\dagger} \hat{a}_{s}$, $\hat{a}_{p}^{ \dagger} \hat{a}_{q}^{\dagger} \hat{a}_{r} \hat{a}_{s}$) 상응하는 Pauli 문자열에 의한 Hamiltonian(예: $\hat{P}_i $, 위 그림 참조). 결과 연산자 세트는 QPU에서 측정할 준비가 되었습니다. 자세한 내용은 [Seeley *et al.* , 2012](https://arxiv.org/abs/1208.5986v1) .

### 능동 공간(Actice space) 선택

일부 분자 시스템의 경우, 화학의 필수적인 대부분의 특징을 포착하기 위해 모든 오비탈을 고려할 필요는 없습니다. 사실, 우리는 화학적 직관을 사용하여 시스템의 에너지를 결정하는 데 가장 중요한 역할을 하는 오비탈의 부분 집합을 선택할 수 있습니다. 이 개념은 <b>활성 공간</b> 선택으로 알려져 있으며 양자 화학 문제의 크기와 비용을 줄일 수 있습니다.

아래 그림에서 볼 수 있듯이, 양자 컴퓨터(왼쪽)에서 모든 오비탈을 시뮬레이션하는 대신 VQE(파란색으로 표시)를 사용하여 양자 컴퓨터에서 시뮬레이션된 일부 오비탈로 활성 공간(오른쪽)을 구성하고, 나머지는 고전적 방법을 사용하여 근사화됩니다(노란색으로 표시).



일반적으로 최적의 활성 공간을 선택하는 것은 어려운 작업입니다. 그러나 적절한 활성 공간의 선택은 믿을 수 있는 파동 함수와 그에 따른 에너지를 얻는 데 너무도 중요합니다.

### Ansätze

화학적 문제에 사용할 수 있는 sätze에는 주로 2가지 유형이 있습니다.

- **q-UCC ansätze** 는 물리적으로 영감을 받아 전자 들뜸상태를 양자 회로에 대략적으로 매핑합니다. q-UCCSD ansatz( Qiskit의 `UCCSD`)는 가능한 모든 단일 및 이중 전자 들뜸을 보유합니다. 쌍을 이루는 이중 q-pUCCD( `PUCCD` ) 및 단일항 q-UCCD0( `SUCCD` )은 이러한 들뜸(매우 짧은 회로를 의미함)의 부분 집합을 고려하고 해리(dissociation) 프로파일에 대해 좋은 결과를 제공하는 것으로 알려져있습니다. 예를 들어, q-pUCCD에는 단일 들뜸이 없고 이중 들뜸이 아래 이미지와 같이 쌍을 이룹니다.
- **Heuristic ansätze( `TwoLocal` )** 는 회로 깊이(circuit depth)를 단축하기 위해 고안 되었지만 여전히 바닥 상태를 계산하는데 사용할 수 있습니다. 아래 그림에서와 같이 R 게이트는 매개변수화된 단일 큐비트 회전을 나타내고 $U_{CNOT}$는 얽힘(2-큐비트 게이트)을 나타냅니다. 주 아이디어는 동일한 블록(독립적인 매개변수 포함)을 특정 $D$로 반복한 후에 바닥 상태에 도달하게 되는 것입니다.

자세한 내용은 [Sokolov *et al.* (q-UCC ansatzes)](https://arxiv.org/abs/1911.10864v2) 및 [Barkoutsos *et al.* (Heuristic ansatzes)](https://arxiv.org/pdf/1805.04340.pdf).

<img src="resources/ansatz.png" width="700/">

### 종합: VQE(Variational Quantum Eigensolver)

고유 상태 $|\psi_{min}\rangle$와 관련된 알 수 없는 최소 고유값 $E_{min}$을 갖는 에르미트 연산자 $\hat H$가 주어지면 VQE는 $E_{min}$로 제한되는 추정치 $E_{\theta}$를 제공합니다:

$$\begin{align*} E_{min} \le E_{\theta} \equiv \langle \psi(\theta) |\hat H|\psi(\theta) \rangle \end{align*}$$

여기서 $|\psi(\theta)\rangle$은 $E_{\theta}$와 관련된 시행(trial) 상태입니다. $U(\theta)$로 표시되는 매개변수화된 회로를 임의의 시작 상태 $|\psi\rangle$에 적용함으로서, 알고리즘은 $|\psi_{min}\rangle$에 대하여 추정값 $U(\theta)|\psi\rangle \equiv |\psi(\theta)\rangle$를 얻습니다. 추정치는 $\theta$ 매개변수를 변경하고 $\langle \psi(\theta) |\hat H|\psi(\theta) \rangle$. 의 기대값을 최소화하여 기존 최적화 프로그램에 의해 반복적으로 최적화됩니다.

바닥 상태 에너지를 찾는 것 외에도 VQE는 분자 동역학 시뮬레이션에 사용할 수 있습니다 [. Sokolov *et al.* , 2021](https://arxiv.org/abs/2008.08144v1) 및 들뜸 상태 계산, [Ollitrault *et al.* , 2019](https://arxiv.org/abs/1910.12890) .

<div class="alert alert-block alert-danger">

 <b>추가적인 세부 사항에 대한 참고 자료</b>
이 알고리즘을 구현하는 Qiskit Nature 튜토리얼은 <a href="https://qiskit.org/documentation/nature/tutorials/01_electronic_structure.html" data-md-type="link">여기</a>를 참조 하십시오 . 자세한 내용은  <a href="https://github.com/Qiskit/qiskit-nature" data-md-type="link">github 저장소 Qiskit Github 첫 페이지</a> &gt;&gt;&gt; 각 기능의 사용에 대한 기본 코드가 있는 <a href="https://github.com/Qiskit/qiskit-nature/tree/main/test" data-md-type="link">테스트 폴더</a> 를 참조하십시오.
</div>

### 양자 운동 방정식(qEOM)

들뜸 에너지는 양자 운동 방정식(qEOM) 알고리즘 [Phys. Rev. Research 2, 043140]을 사용해 계산할 수 있습니다. EOM 방법은 다음 의사 고유값 문제(pseudo-eigen value problem)를 해결하여 들뜸 에너지(바닥 상태와 모든 $n$번째 들뜸 상태 간의 에너지 차이)를 찾습니다.

$$
\begin{pmatrix}
    \text{M} & \text{Q}\\ 
    \text{Q*} & \text{M*}
\end{pmatrix}
\begin{pmatrix}
    \text{X}_n\\ 
    \text{Y}_n
\end{pmatrix}
= E_{0n}
\begin{pmatrix}
    \text{V} & \text{W}\\ 
    -\text{W*} & -\text{V*}
\end{pmatrix}
\begin{pmatrix}
    \text{X}_n\\ 
    \text{Y}_n
\end{pmatrix}
$$

이때

$$
M_{\mu_{\alpha}\nu_{\beta}} = \langle0| [(\hat{\text{E}}_{\mu_{\alpha}}^{(\alpha)})^{\dagger},\hat{\text{H}}, \hat{\text{E}}_{\nu_{\beta}}^{(\beta)}]|0\rangle
$$
$$
Q_{\mu_{\alpha}\nu_{\beta}} = -\langle0| [(\hat{\text{E}}_{\mu_{\alpha}}^{(\alpha)})^{\dagger}, \hat{\text{H}}, (\hat{\text{E}}_{\nu_{\beta}}^{(\beta)})^{\dagger}]|0\rangle
$$
$$
V_{\mu_{\alpha}\nu_{\beta}} = \langle0| [(\hat{\text{E}}_{\mu_{\alpha}}^{(\alpha)})^{\dagger}, \hat{\text{E}}_{\nu_{\beta}}^{(\beta)}]|0\rangle
$$
$$
W_{\mu_{\alpha}\nu_{\beta}} = -\langle0| [(\hat{\text{E}}_{\mu_\alpha}^{(\alpha)})^{\dagger}, (\hat{\text{E}}_{\nu_{\beta}}^{(\beta)})^{\dagger}]|0\rangle
$$

각 행렬의 요소는 해당하는 바닥 상태를 사용하여 양자 컴퓨터에서 측정해야 합니다. Qiskit에서 qEOM을 솔버로 사용하려면 먼저 바닥 상태 계산을 실행하고 결과를 qEOM 솔버에 대한 입력으로 사용해야 합니다.


## 파트 1: 물 분자에 대한 VQE

파트 1에서는 cc-pVDZ 기저로 PySCF 드라이버를 사용하고 Jordan-Wigner 매핑을 사용하여 H$_2$O 분자를 시뮬레이션합니다.

#### 1. 화학 드라이버

Qiskit에서 사용할 수 있는 고전적 화학 코드에 대한 인터페이스를 드라이버라고 합니다. Qiskit Nature는 `PSI4Driver` , `PyQuanteDriver` 및 `PySCFDriver` 와 같은 몇 가지를 지원합니다.

첫 번째 단계는 원자 좌표, 총 스핀(다중도) 및 전하 정보를 지정하여 물 분자를 정의하는 것입니다. 두 번째 단계는 Hartree-Fock 계산을 수행하고 VQE를 수행하는 데 사용할 분자에 대한 모든 필요한 정보를 얻는 것입니다. PySCF 드라이버를 실행하여 두 번째 단계를 구현합니다.

In [ ]:
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureMoleculeDriver, ElectronicStructureDriverType

molecule = Molecule(
    # coordinates are given in Angstrom
    geometry=[
        ["O", [0.0, 0.0, 0.0]],
        ["H", [0.758602, 0.0, 0.504284]],
        ["H", [0.758602, 0.0, -0.504284]]
    ],
    multiplicity=1,  # = 2*spin + 1
    charge=0,
)

driver = ElectronicStructureMoleculeDriver(
    molecule=molecule,
    basis="sto3g",
    driver_type=ElectronicStructureDriverType.PYSCF,
)

properties = driver.run()

<div class="alert alert-block alert-danger">
<p data-md-type="paragraph"><b data-md-type="raw_html">튜토리얼 질문 1</b></p>
<p data-md-type="paragraph">분자의 속성을 살펴보고(https://qiskit.org/documentation/nature/tutorials/08_property_framework.html 참조) 아래 질문에 답하십시오.</p>
<ol data-md-type="list" data-md-list-type="ordered" data-md-list-tight="true">
<li data-md-type="list_item" data-md-list-type="ordered">시스템의 총 전자(알파와 베타) 수는 얼마입니까? (코딩 과제)</li>
<li data-md-type="list_item" data-md-list-type="ordered">분자 및 스핀 오비탈의 수는 얼마입니까? (코딩 과제)</li>
<li data-md-type="list_item" data-md-list-type="ordered">Jordan-Wigner 매핑으로 이 분자를 시뮬레이션하려면 몇 큐비트가 필요합니까? (슬랙에서 토론해 봅시다)</li>
<li data-md-type="list_item" data-md-list-type="ordered">핵 반발 에너지의 값은 얼마입니까? (코딩 과제)</li>
</ol>
</div>

In [ ]:
# WRITE YOUR CODE BETWEEN THESE LINES - START


num_alpha_electrons = 

num_beta_electrons = 

num_spin_orbitals = 

nuclear_rep_energy = 


# WRITE YOUR CODE BETWEEN THESE LINES - END

In [ ]:
from qc_grader.challenges.spring_2022 import grade_ex4a

grade_ex4a(
    num_alpha_electrons=num_alpha_electrons,
    num_beta_electrons=num_beta_electrons,
    num_spin_orbitals=num_spin_orbitals,
    nuclear_rep_energy=nuclear_rep_energy
)

#### 2. 전자 구조 문제(Electronic structure problem)와 활성 공간 변압기 (Active space transformer)

다음 단계는 VQE로 시뮬레이션할 분자 오비탈을 표현하는 활성공간을 선택하고 나머지부분을 고전적인 방법으로 처리하는 것입니다. 활성 공간에 더 많은 오비탈을 포함할수록 양자 회로는 더 많은 계산 비용이 발생합니다. 따라서 목표는 가능한 적은 수의  오비탈을 사용해서 가장 정확한 활성 공간을 구성하는 것입니다. Qiskit Nature의 `ActiveSpaceTransformer` 를 사용하여 활성 공간에 얼마나 많은 전자와 분자 오비탈을 갖고 싶은지와 특정 오비탈 중 어떤 것을 고려하고 싶은지를 지정합니다.

In [ ]:
from qiskit_nature.transformers.second_quantization.electronic import ActiveSpaceTransformer

# Check the occupation of the spin orbitals
PN_property = properties.get_property("ParticleNumber")
print(PN_property)

# Define the active space around the Fermi level 
# (selected automatically around the HOMO and LUMO, ordered by energy)
transformer = ActiveSpaceTransformer(
    num_electrons=2, #how many electrons we have in our active space
    num_molecular_orbitals=2, #how many orbitals we have in our active space
)


# We can hand-pick the MOs to be included in the AS
# (in case they are not exactly around the Fermi level)

# transformer = ActiveSpaceTransformer(
#     num_electrons=2, #how many electrons we have in our active space
#     num_molecular_orbitals=2, #how many orbitals we have in our active space
#     active_orbitals=[4,7], #We are specifically choosing MO number 4 (occupied with two electrons) and 7 (empty)
# )


그런 다음 큐비트(Pauli 문자열)에 매핑하기 전에 페르미온 연산자 목록을 생성하는 `ElectronicStructureProblem` 을 만들 수 있습니다.

In [ ]:
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem

problem = ElectronicStructureProblem(driver, [transformer])

second_q_ops = problem.second_q_ops() # this calls driver.run() internally

hamiltonian = second_q_ops[0]

print(hamiltonian)

<div class="alert alert-block alert-danger">
<p data-md-type="paragraph"><b data-md-type="raw_html">튜토리얼 질문 2</b></p>
<p data-md-type="paragraph">Qiskit을 사용하여 FermionicOperator로 다음 연산자를 생성하고 에르미트인지 확인합니다.</p>
<p data-md-type="paragraph">이 노트의 끝부분에서 답을 찾을 수 있습니다.</p>
</div>

\begin{align*} O = 0.5ia^{\dagger}_1 a_0 + 2.0 a^{\dagger}_2 a_0 \end{align*}

In [ ]:
from qiskit_nature.operators.second_quantization import FermionicOp

# WRITE YOUR CODE BETWEEN THESE LINES - START


# list of valid tuples to initialize the FermionicOp
list_operator = 

# positive integer that represents the length of registers
num_register_length = 


# WRITE YOUR CODE BETWEEN THESE LINES - END

In [ ]:
from qc_grader.challenges.spring_2022 import grade_ex4b

grade_ex4b(list_operator, num_register_length)

#### 3. 페르미온-큐비트 매핑

전자는 페르미온이기 때문에 전자 시스템은 2차 양자화로 표현되는 페르미온 연산자로 구성된 해밀토니언으로 기술됩니다. 양자 컴퓨터는 큐비트로 구성되어 있기 때문에 이러한 페르미온 타입의 해밀토니안을 큐비트 연산자로 변환해야 합니다. 페르미온 연산자를 큐비트 연산자로 변환하는 다양한 유형의 매핑 방법이 있습니다. 다양한 매핑 옵션을 시도할 수 있지만 간단한 대응을 제공하는 `JordanWignerMapper` 를 사용하도록 하겠습니다. 여기에서 하나의 큐비트는 하나의 스핀 오비탈을 나타냅니다.

In [ ]:
from qiskit_nature.mappers.second_quantization import ParityMapper, BravyiKitaevMapper, JordanWignerMapper
from qiskit_nature.converters.second_quantization import QubitConverter

# Setup the mapper and qubit converter
mapper_type = 'JordanWignerMapper'

if mapper_type == 'ParityMapper':
    mapper = ParityMapper()
elif mapper_type == 'JordanWignerMapper':
    mapper = JordanWignerMapper()
elif mapper_type == 'BravyiKitaevMapper':
    mapper = BravyiKitaevMapper()

converter = QubitConverter(mapper)

qubit_op = converter.convert(hamiltonian)
print(qubit_op)

#### 4. 초기 상태

이론 부분에서 설명했듯이 화학에서 좋은 초기 상태는 HF 상태입니다(예: $|\Psi_{HF} \rangle = |0101 \rangle$). 다음과 같이 초기화할 수 있습니다.

In [ ]:
from qiskit_nature.circuit.library import HartreeFock

particle_number = problem.grouped_property_transformed.get_property("ParticleNumber")
num_spin_orbitals = particle_number.num_spin_orbitals
num_particles = particle_number.num_particles


init_state = HartreeFock(num_spin_orbitals, num_particles, converter)
print(init_state)

#### 5. Ansatz

양자 회로 설계에서 가장 중요한 선택 중 하나는 바닥 상태를 근사화하는 데 사용되는 Ansatz를 고르는 것입니다. 다음은 가능한 다른 여러가지 ansatz를 선택할 수 있는  qiskit 회로 라이브러리의  사용 예제 입니다.

In [ ]:
from qiskit.circuit.library import TwoLocal
from qiskit_nature.circuit.library import UCCSD, PUCCD, SUCCD

# Choose the ansatz
ansatz_type = "UCCSD"


# Put arguments for twolocal
if ansatz_type == "TwoLocal":
    # Single qubit rotations that are placed on all qubits with independent parameters
    rotation_blocks = ['ry', 'rz']
    # Entangling gates
    entanglement_blocks = 'cz'
    # How the qubits are entangled 
    entanglement = 'full'
    # Repetitions of rotation_blocks + entanglement_blocks with independent parameters
    repetitions = 1
    # Skip the final rotation_blocks layer
    skip_final_rotation_layer = True
    ansatz = TwoLocal(qubit_op.num_qubits, rotation_blocks, entanglement_blocks, reps=repetitions, 
                      entanglement=entanglement, skip_final_rotation_layer=skip_final_rotation_layer)
    # Add the initial state
    ansatz.compose(init_state, front=True, inplace=True)
elif ansatz_type == "UCCSD":
    ansatz = UCCSD(converter,num_particles,num_spin_orbitals,initial_state = init_state)
elif ansatz_type == "PUCCD":
    ansatz = PUCCD(converter,num_particles,num_spin_orbitals,initial_state = init_state)
elif ansatz_type == "SUCCD":
    ansatz = SUCCD(converter,num_particles,num_spin_orbitals,initial_state = init_state)
elif ansatz_type == "Custom":
    # Example of how to write your own circuit
    from qiskit.circuit import Parameter, QuantumCircuit, QuantumRegister
    # Define the variational parameter
    theta = Parameter('a')
    n = qubit_op.num_qubits
    # Make an empty quantum circuit
    qc = QuantumCircuit(qubit_op.num_qubits)
    qubit_label = 0
    # Place a Hadamard gate
    qc.h(qubit_label)
    # Place a CNOT ladder
    for i in range(n-1):
        qc.cx(i, i+1)
    # Visual separator
    qc.barrier()
    # rz rotations on all qubits
    qc.rz(theta, range(n))
    ansatz = qc
    ansatz.compose(init_state, front=True, inplace=True)

print(ansatz.decompose())

#### 6. 백엔드

양자 회로를 실행하려면 시뮬레이터 또는 실제 양자 컴퓨터인 양자 백엔드를 지정해야 합니다. 우리는 이 챌린지에서 `statevector_simulator`를 사용하겠습니다.

In [ ]:
from qiskit import Aer
backend = Aer.get_backend('statevector_simulator')

#### 7. 옵티마이저

Optimizer는 매개변수의 최적화를 처리합니다. 최적의 최적화 프로그램을 선택하면 필요한 에너지 평가 횟수를 줄이는 데 도움이 될 수 있습니다. 따라서 QPU에서 수행해야 하는 측정 횟수를 정의하는 에너지 수렴을 조사하는 것이 매우 중요합니다.

In [ ]:
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP

optimizer_type = 'L_BFGS_B'

# You may want to tune the parameters 
# of each optimizer, here the defaults are used
if optimizer_type == 'COBYLA':
    optimizer = COBYLA(maxiter=500)
elif optimizer_type == 'L_BFGS_B':
    optimizer = L_BFGS_B(maxfun=500)
elif optimizer_type == 'SPSA':
    optimizer = SPSA(maxiter=500)
elif optimizer_type == 'SLSQP':
    optimizer = SLSQP(maxiter=500)

#### 8. 정확한 eigensolver (고유값 계산기)

정확한 대각화는 행렬로 표현되는 양자 해밀턴의 고유 상태와 에너지 고유값을 결정하는 데 사용되는 수치(numerical) 기술입니다. 이 행렬의 차원은 분자 오비탈의 수에 따라 기하급수적으로 확장되기 때문에 이 방법은 크기가 작은 분자 스템에만 사용할 수 있습니다. 매우 큰 시스템의 경우 파동 함수를 저장하기 위한 메모리가 부족할 것입니다.

사용중인 분자 문제의 크기가 작기 때문에 정확한 대각화를 사용하여 문제를 정확하게 해결할 수 있습니다. 여기에서 얻어지는 정확한 결과는 VQE 결과와 비교할 수 있는 훌륭한 기준값으로 사용됩니다.

In [ ]:
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
import numpy as np 

def exact_diagonalizer(problem, converter):
    solver = NumPyMinimumEigensolverFactory()
    calc = GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result

result_exact = exact_diagonalizer(problem, converter)
exact_energy = np.real(result_exact.eigenenergies[0])
print("Exact electronic energy", exact_energy)
print(result_exact)

# The targeted electronic energy for H2 is -1.85336 Ha
# Check with your VQE result.

#### 9. VQE와 ansatz의 초기 매개변수

이제 VQE 클래스를 가져와 알고리즘을 실행해 봅시다.

In [ ]:
from qiskit.algorithms import VQE
from IPython.display import display, clear_output

# Print and save the data in lists
def callback(eval_count, parameters, mean, std):  
    # Overwrites the same line when printing
    display("Evaluation: {}, Energy: {}, Std: {}".format(eval_count, mean, std))
    clear_output(wait=True)
    counts.append(eval_count)
    values.append(mean)
    params.append(parameters)
    deviation.append(std)

counts = []
values = []
params = []
deviation = []

# Set initial parameters of the ansatz
# We choose a fixed small displacement 
# So all participants start from similar starting point
try:
    initial_point = [0.01] * len(ansatz.ordered_parameters)
except:
    initial_point = [0.01] * ansatz.num_parameters

algorithm = VQE(ansatz,
                optimizer=optimizer,
                quantum_instance=backend,
                callback=callback,
                initial_point=initial_point)

result = algorithm.compute_minimum_eigenvalue(qubit_op)

print(result)

목표는 $\delta E_{chem} = 0.001$ Ha $= 1$ mHa인 화학적 정확도에 가능한 한 효율적으로 도달하는 것입니다. 효율성이란 우리가 다음의 항목들을 최대한 적은 숫자로 사용한다는 것을 의미합니다.

- 매개변수
- 최적화 단계
- CNOT 게이트 수


In [ ]:
# Store results in a dictionary
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller

# Unroller transpile your circuit into CNOTs and U gates
pass_ = Unroller(['u', 'cx'])
pm = PassManager(pass_)
ansatz_tp = pm.run(ansatz)
cnots = ansatz_tp.count_ops()['cx']

energy = result.optimal_value

if ansatz_type == "TwoLocal":
    result_dict = {
        'optimizer': optimizer.__class__.__name__,
        'mapping': converter.mapper.__class__.__name__,
        'ansatz': ansatz.__class__.__name__,
        'rotation blocks': rotation_blocks,
        'entanglement_blocks': entanglement_blocks,
        'entanglement': entanglement,
        'repetitions': repetitions,
        'skip_final_rotation_layer': skip_final_rotation_layer,
        'energy (Ha)': energy,
        'error (mHa)': (energy-exact_energy)*1000,
        '# of parameters': len(result.optimal_point),
        'final parameters': result.optimal_point,
        '# of evaluations': result.cost_function_evals,
        '# of qubits': int(qubit_op.num_qubits),
        '# of CNOTs': cnots}
else:
    result_dict = {
        'optimizer': optimizer.__class__.__name__,
        'mapping': converter.mapper.__class__.__name__,
        'ansatz': ansatz.__class__.__name__,
        'rotation blocks': None,
        'entanglement_blocks': None,
        'entanglement': None,
        'repetitions': None,
        'skip_final_rotation_layer': None,
        'energy (Ha)': energy,
        'error (mHa)': (energy-exact_energy)*1000,
        '# of parameters': len(result.optimal_point),
        'final parameters': result.optimal_point,
        '# of evaluations': result.cost_function_evals,
        '# of qubits': int(qubit_op.num_qubits),
        '# of CNOTs': cnots}

# Plot the results
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1)
ax.set_xlabel('Iterations')
ax.set_ylabel('Energy')
ax.grid()
fig.text(0.7, 0.75, f'Energy: {result.optimal_value:.3f}')
plt.title(f"{result_dict['optimizer']}-{result_dict['mapping']}\n{result_dict['ansatz']}")
ax.plot(counts, values)
ax.axhline(exact_energy, linestyle='--')
fig_title = f"\
{result_dict['optimizer']}-\
{result_dict['mapping']}-\
{result_dict['ansatz']}-\
Energy({result_dict['energy (Ha)']:.3f}).png"
fig.savefig(fig_title, dpi=300)

# Display and save the data
import pandas as pd
import os.path
filename = 'results_h2o.csv'
if os.path.isfile(filename):
    result_df = pd.read_csv(filename)
    result_df = result_df.append([result_dict])
else:
    result_df = pd.DataFrame.from_dict([result_dict])
result_df.to_csv(filename)
result_df[['optimizer','ansatz', 'rotation blocks', 'entanglement_blocks',
    'entanglement', 'repetitions', '# of qubits', '# of parameters', '# of CNOTs', '# of evaluations', 'error (mHa)']]

<div class="alert alert-block alert-danger">
<p data-md-type="paragraph"><b data-md-type="raw_html">튜토리얼 질문 3</b></p>
<p data-md-type="paragraph">모든 매개변수를 실험하고 다음에 답하십시오.</p>
<ol data-md-type="list" data-md-list-type="ordered" data-md-list-tight="true">
<li data-md-type="list_item" data-md-list-type="ordered">효율적인 huristic ansatz를 찾아냈습니까?</li>
<li data-md-type="list_item" data-md-list-type="ordered">
<code data-md-type="codespan">UCCSD, PUCCD or SUCCD</code> ansatz 중 어느 것이 가장 좋습니까?</li>
<li data-md-type="list_item" data-md-list-type="ordered">
<code data-md-type="codespan">UCCSD</code>를 사용했을 때 어떤 optimizer의 성능이 가장 우수했습니까?</li>
</ol>
<p data-md-type="paragraph">가장 효율적인 Ansatz를 찾으려면 정확도, 매개변수의 개수, 최적화 단계의 수, CNOT 게이트 수 등등을 고려해야 합니다. 이것은 토론을 위한 질문이며 채점되지 않습니다. 슬랙의 챌린지 채널에서 마음껏  토론해 봅시다</p>
</div>

## 2부: 물 분자에 대한 qEOM

이 부분에서는 앞서 정의된 물 분자의 들뜸 상태를 계산합니다.

Qiskit에서 qEOM을 솔버로 사용하려면 먼저 바닥 상태 계산을 정의해야 합니다. 이 값을 사용하여 qEOM 솔버를 초기화할 수 있습니다.

In [ ]:
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit_nature.algorithms import GroundStateEigensolver, QEOM, VQEUCCFactory

# This first part sets the ground state solver
# see more about this part in the ground state calculation tutorial
quantum_instance = QuantumInstance(Aer.get_backend("aer_simulator_statevector"))
solver = VQEUCCFactory(quantum_instance)
gsc = GroundStateEigensolver(converter, solver)

# The qEOM algorithm is simply instantiated with the chosen ground state solver
qeom_excited_states_calculation = QEOM(gsc, "sd")

qeom_results = qeom_excited_states_calculation.solve(problem)

print(qeom_results)

<div class="alert alert-block alert-danger">
<p data-md-type="paragraph"><b data-md-type="raw_html">튜토리얼 질문 4</b></p>
<p data-md-type="paragraph">정확한 들뜸 상태 에너지를 얻고 qEOM에서 얻은 에너지와 비교합니다 (코딩 과제). 관찰된 차이점에 대해 슬랙에서 토론해 봅시다.</p>
</div>

In [ ]:
# WRITE YOUR CODE BETWEEN THESE LINES - START



# WRITE YOUR CODE BETWEEN THESE LINES - END

## 추가 정보

**만든 사람:** Ieva Liepuoniute 와 Pauline Ollitrault

**한글 번역:** 김경훈, 강다영, 신소영

**버전:** qiskit-nature 0.3.2